In [1]:
%install-location $cwd/swift-install
%install '.package(path: "$cwd/SchwiftyNotebook_Annealing")' SchwiftyNotebook_Annealing

Installing packages:
	.package(path: "/home/ubuntu/workspace/fast-ai-swift/SchwiftyNotebook_Annealing")
		SchwiftyNotebook_Annealing
With SwiftPM flags: []
Working in: /tmp/tmpdbpjeqce/swift-install
/home/ubuntu/swift/usr/bin/swift: /home/ubuntu/anaconda3/envs/swift-env/lib/libuuid.so.1: no version information available (required by /home/ubuntu/swift/usr/bin/swift)
/home/ubuntu/swift/usr/bin/swift: /home/ubuntu/anaconda3/envs/swift-env/lib/libuuid.so.1: no version information available (required by /home/ubuntu/swift/usr/bin/swift)
/home/ubuntu/swift/usr/bin/swift: /home/ubuntu/anaconda3/envs/swift-env/lib/libuuid.so.1: no version information available (required by /home/ubuntu/swift/usr/bin/swift)
/home/ubuntu/swift/usr/bin/swift: /home/ubuntu/anaconda3/envs/swift-env/lib/libuuid.so.1: no version information available (required by /home/ubuntu/swift/usr/bin/swift)
/home/ubuntu/swift/usr/bin/swift: /home/ubuntu/anaconda3/envs/swift-env/lib/libuuid.so.1: no version information availabl

In [2]:
//export
import Path
import TensorFlow
import SchwiftyNotebook_Annealing

In [3]:
//export
public protocol TeachableModel {
    
}

In [4]:
(1...10).reduce(into: 0) { $0 + $1}

0


In [11]:
//export
public struct Teacher {
    
    
    public func teachModel(model: TeachableModel,
                           numberOfEpochs: Int) -> TeachableModel {
        // todo: Notify pre teach
        
        let taughtModel = (1...numberOfEpochs)
            .reduce(into: model) { (currentModel: inout TeachableModel, 
                                    epoch: Int) in
                                  
                let trainedModel = trainModel(model: currentModel,
                                              epoch: epoch)
                currentModel = validateModel(model: model,
                                     epoch: epoch)
            }
        
        // todo: Notify post taught
        
        return taughtModel
    }
    
    private func trainModel(model: TeachableModel, epoch: Int) -> TeachableModel {
        return model
    }
    
    private func validateModel(model: TeachableModel, epoch: Int) -> TeachableModel {
        return model
    }
    
}